In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time

In [2]:
db_connection = sqlite3.connect("../database/DAOGovernance.db")
cursor = db_connection.cursor()

In [3]:
driver = webdriver.Firefox()
driver.get("https://messari.io/governor/daos")


def scroll(num):
	for _ in range(0,num):
		driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
		time.sleep(4)


def main():
	WebDriverWait(driver, 100).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, 'div.jss26:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > button:nth-child(2)'))).click()
	for i in range(1,100):
		WebDriverWait(driver, 100).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, f".MuiTableBody-root > tr:nth-child({i}) > td:nth-child(1) > a:nth-child(1)"))).click()
		WebDriverWait(driver, 100).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, f".css-kmnzss > div:nth-child(2)")))
		
		time.sleep(4)
		get_tag_and_type()
		
		if i > 20:
			scroll()
			get_tag_and_type()
		
		elif i > 40:
			scroll(2)
			get_tag_and_type()

		time.sleep(10)
		driver.get("https://messari.io/governor/daos")
	
	db_connection.close()

def find_dao_helper(soup):
	gov_div = soup.findAll("div", class_='MuiBox-root css-j7qwjs')[2]
	gov_div_children = gov_div.findChildren("div")
	name_and_price = [i.string for i in gov_div_children][0]
	dao_name = name_and_price.split()[0]
	return dao_name


def get_tag_and_type():
	
	soup = BeautifulSoup(driver.page_source, "html.parser")
	dao_name = find_dao_helper(soup)

	type_div = soup.find("div", class_='MuiBox-root css-1qocved')
	dao_type = type_div.findChildren("span")[0].string

	tag_div = soup.find("div", class_='MuiBox-root css-j7qwjs')
	dao_tag = tag_div.findChildren("span")[0].string


	l = soup.find('div', class_='MuiBox-root css-1wvvhxi')
	k = l.findChildren('div', class_='MuiBox-root css-1qocved')
	print([i.string for i in k])
	
	# dao_table = soup.find("tbody", class_="MuiTableBody-root css-fo7z8p")
	# dao_row = dao_table.findChildren("tr", class_="MuiTableRow-root css-1hifdwt")

	# type_list = ['Social / Community', 'Investment', 'Protocol', 'Product', 'Service', 'Grants', 
	# 'Media', 'Impact', 'Collector']

	# i = 0 

	# for dao in dao_row:
	# 	raw_data = dao.findChildren("span")
	# 	data_list = [i.string for i in raw_data]
	# 	dao_type = data_list[1]
	# 	if data_list[2] in type_list:
	# 		dao_tag = data_list[3]
	# 	else:
	# 		dao_tag = data_list[2]
	# 	select_query = f"SELECT * FROM Governance WHERE DAO = '{dao_name}';"
	# 	cursor.execute(select_query)
	# 	results = cursor.fetchall()
	# 	if results:  # If the DAO exists, update its values in the table
	# 		insert_vals = f"""
	# 			UPDATE Governance SET
	# 			`Type`= '{dao_type}',
	# 			`Tag`= '{dao_tag}'
	# 			WHERE DAO='{dao_name}';
	# 			"""
	# 		cursor.execute(insert_vals)
	# 		db_connection.commit()
	# 		print(dao_name)
	
		

main()
driver.quit()

['Protocol', 'DeFi']
